<a href="https://colab.research.google.com/github/FarahGamal/Digital-Filter-Design/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
# installation
# !pip install tensorflow opencv-python matplotlib

# Imports and Attributes Initialization

In [2]:
#imports
import os                              
import cv2
import imghdr
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import svm
from sklearn.svm import SVC
from keras.models import load_model
from matplotlib import pyplot as plt
from keras.applications import ResNet50
from sklearn.preprocessing import normalize
from tensorflow.keras.preprocessing import image
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, roc_curve, roc_auc_score, ConfusionMatrixDisplay, classification_report

In [3]:
# Attributes Initialization
data_dir = '/content/drive/MyDrive/Dataset_image_project/data' 
SVM_dir = '/content/drive/MyDrive/Dataset_image_project/model/svm_model50.pkl'
features_dir = '/content/drive/MyDrive/Dataset_image_project/model/features.npy'
labels_dir = '/content/drive/MyDrive/Dataset_image_project/model/labels.npy'
image_exts = ['jpeg','jpg', 'bmp', 'png']                               #putting all the wanted image extensions
class_names = ['Drusen', 'Exudate', 'Normal']

# Setup 

In [4]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:                                                  
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
tf.config.list_physical_devices('GPU')

[]

# Remove dodgy images

We loop over the dataset to remove dodgy images.
we load the 3 classes that we want Normal, Exudate and drusen, then check for any corrupted image to remove it.

In [6]:
# we loop over the dataset to remove dodgy images
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
          # we load the 3 classes that we want Normal, Exudate and drusen                                                                 
          img = cv2.imread(image_path)                                     
          tip = imghdr.what(image_path)
          # if condition to remove any corrupted image
          if tip not in image_exts: 
              print('Image not in ext list {}'.format(image_path))
              os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# Load and Scale Data

We first define an ImageDataGenerator object and set rescale to 1./255 to rescale the pixel values to the range [0, 1]. Then we use the flow_from_directory method of the ImageDataGenerator object to load a subset of the dataset. We set the batch_size, the target_size, and the interpolation method to be 32, 224x244 and 'bilinear'.

In [7]:
# we define an ImageDataGenerator object and 
# set rescale to 1./255 to rescale the pixel values to the range [0, 1].
data_generator = ImageDataGenerator(rescale=1./255)

# we use the flow_from_directory method of the 
# ImageDataGenerator object to load a subset of the dataset
# we made resizing and image interpolation using bilinear
train = data_generator.flow_from_directory(
    directory=data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False,
    interpolation='bilinear'
)

Found 172 images belonging to 3 classes.


# Feature Extraction and Split Data

we first load the pre-trained ResNet50 model to extract features from images and store them in arrays.
then we save the features and labels in .npy file the load them to split the images into train and test.

In [8]:
# Load the pre-trained ResNet50 model
ResNet50_model = ResNet50(weights='imagenet', include_top=False)

# Extract features from the images using ResNet50 and store them in arrays
train_features = ResNet50_model.predict(train, steps=len(train), verbose=1)

6/6 [==============================] - 55s 9s/step


In [9]:
# Save the preprocessed features and labels from the images
np.save(features_dir, train_features)
np.save(labels_dir, train.classes)

In [10]:
# Load the preprocessed features and labels from the images
features = np.load(features_dir)
labels = np.load(labels_dir)

In [11]:
# Split image into train and test
feature_train, feature_test, label_train, label_test = train_test_split(features, labels, test_size=0.1, random_state=42)

# Build Deep Learning Model

After we extract the features from the images using ResNet50 and store them in arrays, we then flatten and use to train an SVM classifier.

In [12]:
# Flatten the features
feature_train = np.reshape(feature_train, (feature_train.shape[0], -1))
label_test = np.reshape(label_test, (label_test.shape[0], -1))

# Define the SVM model
svm_model = SVC(kernel='linear', C=1, probability=True)

# Train the SVM model on the features
svm_model.fit(feature_train, label_train)

SVC(C=1, kernel='linear', probability=True)

In [13]:
# Save SVM model
svm_model = OneVsRestClassifier(SVC(kernel='linear', probability=True))  ##malosh lzma i think
svm_model.fit(feature_train, label_train)                                ##w dah kaman :)
joblib.dump(svm_model, SVM_dir)

['/content/drive/MyDrive/Dataset_image_project/model/svm_model50.pkl']

In [14]:
# Load SVM model
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
svm_model = joblib.load(SVM_dir)

# Test Model

we test the model on the test features and show results of actual and predicted.

In [15]:
# Evaluate the SVM model on the test features
predicted_class = []
feature_test = np.reshape(feature_test, (feature_test.shape[0], -1))
for i in range(len(feature_test)):
  svm_preds = svm_model.predict(feature_test)[i]
  predicted_class.append(svm_preds)

In [16]:
# Show actual and predicted
test_result = pd.DataFrame({'Actual' : label_test.flatten(), 'Predicted' : np.array(predicted_class)})
test_result['Actual'] = test_result['Actual'].replace(0, 'Drusen')
test_result['Actual'] = test_result['Actual'].replace(1,'Exudate')
test_result['Actual'] = test_result['Actual'].replace(2,'Normal')

test_result['Predicted'] = test_result['Predicted'].replace(0, 'Drusen')
test_result['Predicted'] = test_result['Predicted'].replace(1,'Exudate')
test_result['Predicted'] = test_result['Predicted'].replace(2,'Normal')
test_result

,Actual,Predicted
0,Exudate,Exudate
1,Normal,Normal
2,Normal,Normal
3,Drusen,Drusen
4,Normal,Normal
5,Normal,Normal
6,Exudate,Exudate
7,Normal,Normal
8,Exudate,Exudate
9,Drusen,Drusen


# Evaluate Model

Here we evaluate the model by calculating accuracy, precision, sensitivity, f1 score, specificity, and AUC.

In [17]:
# calulate specificity
def specificity(y_true, y_pred, class_index):
    cm = confusion_matrix(y_true, y_pred)
    tn = sum(cm[i, j] for i in range(cm.shape[0]) for j in range(cm.shape[1]) if i != class_index and j != class_index)
    fp = sum(cm[i, j] for i in range(cm.shape[0]) for j in range(cm.shape[1]) if i != class_index and j == class_index)
    return tn / (tn + fp)

In [18]:
# calculate accuracy 
def accuracy_per_class(y_true, y_pred, class_index):
    cm = confusion_matrix(y_true, y_pred)
    tn = sum(cm[i, j] for i in range(cm.shape[0]) for j in range(cm.shape[1]) if i != class_index and j != class_index)
    fp = sum(cm[i, j] for i in range(cm.shape[0]) for j in range(cm.shape[1]) if i != class_index and j == class_index)
    tp = cm[class_index, class_index]
    fn = sum(cm[i, class_index] for i in range(cm.shape[0]) if i != class_index)
    return (tp + tn) / (tp + tn + fp + fn)

In [ ]:
confusion = confusion_matrix(label_test, predicted_class)
print('\nClassification Report\n')
print(classification_report(label_test, predicted_class, target_names=['Drusen', 'Exudate', 'Normal']))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix = confusion, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [21]:
# Accuracy
score = accuracy_per_class(label_test, predicted_class, 0)
print("Accuracy for Drusen: {:.2f}".format(score))
score = accuracy_per_class(label_test, predicted_class, 1)
print("Accuracy for class Exudate: {:.2f}".format(score))
score = accuracy_per_class(label_test, predicted_class, 2)
print("Accuracy for class Normal: {:.2f}".format(score))

# Specificity
score = specificity(label_test, predicted_class, 0)
print("Specificity for Drusen: {:.2f}".format(score))
score = specificity(label_test, predicted_class, 1)
print("Specificity for class Exudate: {:.2f}".format(score))
score = specificity(label_test, predicted_class, 2)
print("Specificity for class Normal: {:.2f}".format(score))

# AUC
test = np.reshape(label_test, (-1, 1))
predicted = np.reshape(predicted_class, (-1, 1))
predicted = normalize(predicted, axis=1, norm='l1')
test = normalize(test, axis=1, norm='l1')
auc = roc_auc_score(test, predicted, multi_class='ovo')
print("AUC: {:.2f}".format(auc))

Accuracy for Drusen: 1.00
Accuracy for class Exudate: 1.00
Accuracy for class Normal: 1.00
Specificity for Drusen: 1.00
Specificity for class Exudate: 1.00
Specificity for class Normal: 1.00
AUC: 1.00


# Classification

In [22]:
# Classify image
from tensorflow.keras.preprocessing import image

# Load and preprocess the image
img_path = '/content/drive/MyDrive/Dataset_image_project/data/Exudate/P003010_Img006477_Ex003485_Dx05_R3_GM_QA964_Fa_OS.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Extract features
resnet_features = resnet_model.predict(x)
# Flatten the features
resnet_features_flat = np.reshape(resnet_features, (resnet_features.shape[0], -1))
# Predict the classes using the SVM model
svm_pred = svm_model.predict(resnet_features_flat)

class_names = ['Drusen', 'Exudate', 'Normal']
print('Predicted class:', class_names[svm_pred[0]])

1/1 [==============================] - 2s 2s/step
Predicted class: Normal


In [23]:
svm_pred

array([2], dtype=int32)